We are curious as to whether our subset of COS spectra were targeting High Velocity Clouds (HVC) or the Magellanic Stream (MS). If they were, this set is biased at some level. What is that level of bias?

In [6]:
import numpy as np
import os,sys
import datetime
from scipy import misc
%matplotlib inline
from matplotlib import pyplot as plt
import pickle as pk
from astropy.io import fits
from astroquery import mast
from astropy import coordinates
from astropy import time
from astropy import units

Load the objects from Yong Zheng's List of Si IV absorbers

In [7]:
objnames = np.load('/Users/jegpeek/Dropbox/Zheng_CGQALD/SiIV-QSO.sav.npy')

What are the objnames?

In [14]:
objnames

array([['MRK1392', 'PG1352+183', 'RXS-J21388-3828', 'CTS487',
        'QSO-B2356-309', 'PKS2155-304', '1ES1553+113',
        'SDSS-J141038.39+230447.1', 'NGC-5548', 'QSO-B1617+1731',
        'PG-1407+265', 'MARK509', 'SDSSJ133045.15+281321.4', 'MR2251-178',
        'PHL1811', 'HB89.1259+281', 'RXJ2139.7+0246', 'QSO-B1440+3539',
        'MRK1513', 'PMNJ2345-1555', 'SDSSJ135341.03+361948.0', 'MRK304',
        'SDSSJ232259.98-005359.2', 'HS1831+5338', 'NGC-7469', 'PG1626+554',
        'X-CYG-X-2', 'SBS1624+575', 'SDSS.J130429.04+311308.2',
        'SBS1527+564', 'MRK290', '4C--01.61', 'HS1700+6416', 'PG1309+355',
        'MRK876', 'MRK817', 'FBS1526+659', 'RXSJ15032+6809', 'RBS2055',
        'PG0003+158', 'MRK-335', 'SDSSJ145907.58+714019.9', 'PG1351+640',
        '4C25.01', 'IRASF00040+4325', 'QSO-B0026+129', 'PG1259+593',
        'RXSJ00437+3725', 'RXSJ00508+3536', 'MRK1148', 'RXSJ00537+2232',
        'IZW1', 'PG0052+251', 'SDSSJ123335.07+475800.4', 'NGC-3516',
        '3C48', '3C263', 

Now go search the archive for all the proposal ids that are in each target

In [9]:
#sorry, I am bad at python :)
PIDs = np.array(0)
targets = np.array(0)

for i, oname in enumerate(objnames[0]):
    table_obj = mast.Observations.query_criteria(target_name=oname, instrument_name='COS/FUV')
    tcoord = coordinates.SkyCoord(table_obj[0]["s_ra"],table_obj[0]["s_dec"], unit='deg')
    table = mast.Observations.query_criteria(coordinates=tcoord, radius=2*units.arcsecond, instrument_name='COS/FUV')
    PIDs = np.append(PIDs, table["proposal_id"].data)
    targets = np.append(targets, np.repeat(oname, len(table)))
PIDs = PIDs[1:]
targets = targets[1:]
abstype = np.array(len(PIDs))

How many targets?

In [10]:
len(targets)

2286

And how many unique IDs? We are going to feed this to: https://archive.stsci.edu/hst/abstract.html

In [11]:
uPIDs = np.unique(PIDs)
print(",".join(uPIDs))

11505,11520,11524,11528,11541,11598,11619,11686,11692,11698,11728,11741,11834,12022,12024,12025,12029,12038,12204,12212,12248,12275,12276,12486,12536,12569,12604,12612,12863,12864,12916,12936,12953,12958,12988,13008,13033,13115,13184,13301,13330,13347,13382,13383,13398,13423,13429,13444,13448,13451,13491,13651,13749,13774,13811,13812,13814,13833,13836,13840,13852,14054,14071,14176,14242,14268,14479,14481


Now we want to go through each abstract and figure out if they are targeting HVCs or the Mag Stream. We just do a search on the resulting webpage for HVC and MS.

By my read these are the culprits:

HVCs: 12275 13840

MS: 12204 12604

There are now two questions: what targets were observed by HVC and MS programs and what targets were *only* observed by MS and HVC programs. Turns out the answer is almost the same: **13 or 14 of the targets.**

In [92]:
whpidmshvc = np.where((PIDs == '12275') | (PIDs == '13840')| (PIDs == '12204')| (PIDs == '12604'))

In [93]:
whpid_not_mshvc = np.where((PIDs != '12275') & (PIDs != '13840')& (PIDs != '12204')& (PIDs != '12604'))

In [94]:
observed_by_non_HVCMS_programs  = np.unique(targets[whpid_not_mshvc])
utargs = np.unique(targets)

In [95]:
temp3 = [x for x in utargs if x not in observed_by_non_HVCMS_programs]

In [96]:
print(temp3)

['B0117-2837', 'B0120-28', 'ESO292-G24', 'FAIRALL9', 'HE0056-3622', 'HS1831+5338', 'PG1112+431', 'PKS1136-13', 'RBS1892', 'RXJ2139.7+0246', 'RXSJ09565-0452', 'TON1187', 'TONS210']


In [97]:
print(np.unique(targets[whpidmshvc]))

['B0117-2837' 'B0120-28' 'ESO292-G24' 'FAIRALL9' 'HE0056-3622'
 'HS1831+5338' 'MRK304' 'PG1112+431' 'PKS1136-13' 'RBS1892'
 'RXJ2139.7+0246' 'RXSJ09565-0452' 'TON1187' 'TONS210']


This means, that of our 138 observations, about 10% were targeted through HVCs. We can show that our final results are not strongly changed by including or excluding these.